[![Notebook Tutorial](__code/__all/notebook_tutorial.png)](https://neutronimaging.pages.ornl.gov/tutorial/notebooks/extract_sans_reductionlog_metadata)

<img src='__docs/__all/notebook_rules.png' />

# Select Instrument

In [1]:
from __code.sans import extract
from __code.__all import custom_style
custom_style.style()

initializer = extract.Initializer()
initializer.select_instrument()

# Select your ReductionLog files and then the Metadata to extract 

In [2]:
working_dir = initializer.get_working_dir()
o_extract = extract.Extract(working_dir=working_dir,
                           instrument=initializer.get_instrument())
o_extract.select_reductionlog()

Label(value='Command + Click: to select more than 1 element in the right widget')

# Select output folder

In [3]:
o_extract.export()

entry_path: ['reduction_information', 'sample_logs', 'main', 'CryoSampleReadback']
value: [0. 0.]
entry_path: ['reduction_information', 'sample_logs', 'main', 'CryoVTI']
value: [0. 0.]
entry_path: ['reduction_information', 'sample_logs', 'main', 'CryoVTIReadback']
value: [0. 0.]
entry_path: ['reduction_information', 'sample_logs', 'main', 'CryoValve']
value: [0. 0.]
entry_path: ['reduction_information', 'sample_logs', 'main', 'DLambdaReq']
value: [0.132 0.132]
output file will be /HFIR/CG2/s1mid_q_reduction_log_extracted.txt


# Debugging 

In [7]:
import os
import h5py

In [6]:
nexus_name = '/Users/j35/IPTS/CG2/s1mid_q_reduction_log.hdf'
assert os.path.exists(nexus_name)

In [35]:
path = ['reduction_information', 'sample_logs', 'main', 'CryoVTI']

In [46]:
with h5py.File(nexus_name, 'r') as nxs:
    nxs = nxs.get('reduction_information')
    nxs = nxs.get('sample_logs')
    nxs = nxs.get('main')
    nxs = nxs.get('CryoVTI')
    print(nxs[()])


[0. 0.]
